In [1]:
import pandas as pd

In [65]:
from genderize import Genderize

[{'name': 'James', 'gender': 'male', 'probability': 0.99, 'count': 6359}, {'name': 'Eva', 'gender': 'female', 'probability': 1.0, 'count': 1715}, {'name': 'Thunderhorse', 'gender': None}]


In [77]:
df = pd.read_csv('./data/Shark Tank Companies.csv').dropna(subset=['entrepreneurs'])

In [67]:
names = [name.split(',')[0].split(' ')[0] for name in df.entrepreneurs]
# names = list(filter(lambda x: x.split(',')[0],names))
names

['Darrin',
 'Tod',
 'Tiffany',
 'Nick',
 'Kevin',
 'Susan',
 'Craig',
 'Mary',
 'Robert',
 'Mark',
 'Marian',
 'Brian',
 'Lori',
 'Jeff',
 'Ken',
 'Paul',
 'Amy',
 'Dan',
 'Kwami',
 'Gina',
 'Rodolfo',
 'Jeff',
 'Cactus',
 'Rikki',
 'Irena',
 'Kimberly',
 'Jonathan',
 'David',
 'Heath',
 'Buck',
 'Dedelle',
 'Todd',
 'Leslie',
 'Sawyer',
 'Edwin',
 'Danon',
 'D.J.',
 'Lisa',
 'Gayla',
 'Ray',
 'Dan',
 'Dr.',
 'Jeff',
 'Dr.',
 'Brian',
 'Sandy',
 'Andy',
 'Michael',
 'Dorene',
 'Alfonzo',
 'Marix',
 'Nate',
 'Tim',
 'Anthony',
 'Kimberly',
 'Jill',
 'Tim',
 'Jeff',
 'Michael',
 'Mark',
 'Alan',
 'Aida',
 'Michele',
 'Nicole',
 'Jonathan',
 'Megan',
 'James',
 'Shawn',
 'Nikki',
 'Matty',
 'Joe',
 'Brian',
 'Stacy',
 'Kiersten',
 'Emily',
 'James',
 'Mike',
 'Darryl',
 'Ken',
 'Johnny',
 'April',
 'George',
 'Bob',
 'Jan',
 'Shane',
 'Melissa',
 'Kim',
 'Deidrea',
 'Chris',
 'Stuart',
 'Les',
 'Andy',
 'Johnson',
 'Julie',
 'Jason',
 'Jeff',
 'Rebecca',
 'Kim',
 'Dominique',
 'Aldo',
 'D

### Determine gender of entrepreneur based on first name

In [70]:
genders = [(item['name'], item['gender']) for item in Genderize().get(names)]
genders[:10]

[('Darrin', 'male'),
 ('Tod', 'male'),
 ('Tiffany', 'female'),
 ('Nick', 'male'),
 ('Kevin', 'male'),
 ('Susan', 'female'),
 ('Craig', 'male'),
 ('Mary', 'female'),
 ('Robert', 'male'),
 ('Mark', 'male')]

In [78]:
df['gender'] = [g[1] for g in genders]

In [94]:
((df.gender=='male').values * df.deal.values).sum()

128

In [99]:
g = 'male'
percent_funded = ((df.gender==g).values * df.deal.values).sum()/(df.gender==g).sum()
print("{:.1f} % of {}s were funded.".format(100*percent_funded, g))

48.7 % of males were funded.


In [100]:
g = 'female'
percent_funded = ((df.gender==g).values * df.deal.values).sum()/(df.gender==g).sum()
print("{:.1f} % of {}s were funded.".format(100*percent_funded, g))

56.0 % of females were funded.


In [79]:
df.head()

,deal,description,episode,category,entrepreneurs,location,website,askedFor,exchangeForStake,valuation,season,shark1,shark2,shark3,shark4,shark5,title,episode-season,Multiple Entreprenuers,gender
0,False,Bluetooth device implant for your ear.,1,Novelties,Darrin Johnson,"St. Paul, MN",NaN,1000000,15,6666667,1,Barbara Corcoran,Robert Herjavec,Kevin O'Leary,Daymond John,Kevin Harrington,Ionic Ear,1-1,False,male
1,True,Retail and wholesale pie factory with two reta...,1,Specialty Food,Tod Wilson,"Somerset, NJ",http://whybake.com/,460000,10,4600000,1,Barbara Corcoran,Robert Herjavec,Kevin O'Leary,Daymond John,Kevin Harrington,Mr. Tod's Pie Factory,1-1,False,male
2,True,Ava the Elephant is a godsend for frazzled par...,1,Baby and Child Care,Tiffany Krumins,"Atlanta, GA",http://www.avatheelephant.com/,50000,15,333333,1,Barbara Corcoran,Robert Herjavec,Kevin O'Leary,Daymond John,Kevin Harrington,Ava the Elephant,1-1,False,female
3,False,"Organizing, packing, and moving services deliv...",1,Consumer Services,"Nick Friedman, Omar Soliman","Tampa, FL",http://collegehunkshaulingjunk.com/,250000,25,1000000,1,Barbara Corcoran,Robert Herjavec,Kevin O'Leary,Daymond John,Kevin Harrington,College Foxes Packing Boxes,1-1,False,male
4,False,Interactive media centers for healthcare waiti...,1,Consumer Services,Kevin Flannery,"Cary, NC",http://www.wispots.com/,1200000,10,12000000,1,Barbara Corcoran,Robert Herjavec,Kevin O'Leary,Daymond John,Kevin Harrington,Wispots,1-1,False,male
